  
<span style="color: #a2cf2a; font-family: Babas; font-size: 3em;">
Preparación del Dataset  
para la API GeoRef
</span>   

Ezequiel Bernatene  
eb.modernizacion@gmail.com  


Se presenta como objetivo la construcción de un dataset que facilite la evaluación de la API GeoRef, como así también el desarrollo de herramientas que mejoren la performance de la misma. Se construye una metodología para pasar una dirección escrita en un formato no estructurado a otro estructurado. Para esto se plantea una estratégia de segmentación de la información presente en el string de entrada.  
Para la construcción del dataset que permita testear diferentes implementaciones de la API se hace uso de diversos datasets obtenidos de situaciones reales concretas, los cuales presentan los campos mínimos para poder usar en la API GeoRef o que es posible inferirlos. 

## Funciones Auxiliares

* [Normalización de Strings](#fnorm)
* [Segmentación por calles](#fcalle)
* [Segmentación de numeración](#fnumer)
* [Segmentación de Unidades en la Numeración](#unidades)
* [Normalización de Direcciones](#norm)




## Tablas Auxiliares
* [Codigo BAHRA](#bahra)  
* [Codigo Caracteristicas Telefonicas](#codigo_tel)

## Lista de Datasets

* [data-sip.csv](#data-sip)
* [PARANA - Consolidado Dia 1 y 2.xlsx](#PARANA)
* [RIO IV - Consolidado Dia 1 y 2.xlsx](#RIO IV)
* [FORTALECIMIENTO REPI.xlsx](#FORTALECIMIENTO)
* [500_CASOS_A_NORMALIZAR.xls](#500 casos)
* [500_CASOS_GEOCONFIRMADOS.xls](#500 casos geo)
* [CUS-medicamentos.xlsx](#CUS)
* [refes_centros_publicos_de_atencion_primaria_22-06-2018.xlsx](#CPAP)
* [refes_establecimientos_de_salud_privados_22-06-2018.xlsx](#ESP)
* [refes_establecimientos_de_salud_publica_con_internacion_22-06-2018.xlsx](#ESPI)
* [refes_registros_federal_de_establecimientos_de_salud_22-06-2018.xlsx](#RFES)

<span style="color: #a2cf2a; font-family: Babas; font-size: 3em;">
Funciones Auxiliares
</span>
<a class=anchor id=func></a>

In [1]:
import pandas as pd
import re
from unicodedata import normalize

/home/ezequiel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Normalización de String
<a class='anchor' id='fnorm'></a>  

Esta función es la encargada de eliminar todo tipo de acento del _string_, pasandolo a formato _Uppercase_. La función no elimina las letras _"Ñ"_.

In [2]:
def strip_accents(string=None):
    # -> NFD y eliminar diacríticos
    s = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
               normalize( "NFD", string), 0, re.I)
    # -> NFC
    return normalize( 'NFC', s).upper()

In [3]:
# Test de función 'strip_accents()'
s = strip_accents('NoréstÉñÃ, N°º âgüìta')
s

'NORESTEÑA, N°º AGUITA'

In [5]:
s = "how much for th.e/ mapÑOSÃˆ PASOS le syrup? $20.99? Thát's ridiculous!!!"
re.sub(r'[^\w]', ' ', s)

'how much for th e  mapÑOSÃˆ PASOS le syrup   20 99  Thát s ridiculous   '

### Segmentación por calles.
<a class='anchor' id='fcalle'></a>  

Esta función interpreta que los posibles formatos en los que puede pasarse una dirección son tres:  
* _calle1_ 
* _calle1_ __"Y"__ _calle2_
* _calle1_ __"ENTRE"__ _calle2_ __"Y"__ _calle3_  

La misma interpreta el formato en que se encuentra el _string_ y realiza la segmentación correspondiente. En cualquiera de los casos puede encontrarse incluida la numeración correspondiente la cual no será segmentada por esta función, quedando incluida dentro de las segmentaciones _calle1_, _calle2_ ó _calle3_. La función interpreta diversas formas de escribir __"ENTRE"__ ("ENT", "E/", "E"), como así también el caso en que __"Y"__ deba ser reemplazado por __"E"__, ej. "Jujuy __"E"__ Independencia".

In [6]:
def split_streets(string):
    string = string.strip()
    string = re.sub(r'(?=[0-9])(?<=[A-Z])', r' ', string)
    string = re.sub(r'(?<=[0-9])(?=[A-Z])', r' ', string)
    streets = re.compile(r'([A-Z0-9\s°º,.]+)\s(E|E/|ENT|ENTRE)\s([A-Z0-9\s.]+)\sY\s([A-Z0-9\s°º.]+)')
    # Esta expresion regular no contempla el caso "calle1 ####, esquina Calle2"
    result = streets.search(string=string)
    if result:
        calle1 = result.groups()[0]
        calle2 = result.groups()[2]
        calle3 = result.groups()[3]
    else:
        streets = re.compile(r'([A-Z0-9\s°º.]+)\sY\s([A-Z0-9\s°º.]+)')
        result = streets.search(string=string)
        if result:
            calle1, calle2 = result.groups()
            calle3 = None
        else:
            calle1 = string
            calle2 = None
            calle3 = None
    return calle1, calle2, calle3

In [7]:
# Test de la función 'split_streets()'
string = ['BELGRANO Y ENTRE RIOS','RUTA 8 KM 52','F.ALCORTA Y PAMPA','SAN JUAN DE DIOS 666, ENT BELGRANO Y ENTRE RIOS',
          'YATAY 541', 'CALLE 41 12 E/ 1 Y 2']

In [8]:
for s in string:
    print(split_streets(s))

('BELGRANO', 'ENTRE RIOS', None)
('RUTA 8 KM 52', None, None)
('F.ALCORTA', 'PAMPA', None)
('SAN JUAN DE DIOS 666,', 'BELGRANO', 'ENTRE RIOS')
('YATAY 541', None, None)
('CALLE 41 12', '1', '2')


### Segmentación de numeración.
<a class='anchor' id='fnumer'></a>

Esta función extrae, en caso de que posea, la numeración presente en las segmentaciones de la dirección producidas por la función _split_streets()_. la misma contempla la posibilidad de que se de información adicional (Edificio, departamento) evitando incluila en la segmentación. A su vez, preserva la información donde se resalta si el número indicado corresponde a una numeración o altura en kilometros.  
En los casos donde no encuentre numeración, la función devuelve _None_.

In [9]:
def numeracion(string):
    numer = re.compile('([\s][N°ºKM]{,2})[\s]{0,}([0-9]+)')

    delete = ['CALLE','AVENIDA','AV.','AV','RUTA','NACIONAL','NAC.','NAC','PROVINCIAL','PROV.','PROV',
              'DIAGONAL','DIAG.','DIAG','BOULEVARD','BV.','BV'] # Esta parte es complicada.... No me gusta
    
    for word in delete:
        try:
            string = string.replace(word,'')
        except:
            pass
    try:
        string = string.strip()
        string = re.sub(r'(?=[0-9])(?<=[A-Z])', r' ', string)
        string = re.sub(r'(?<=[0-9])(?=[A-Z])', r' ', string)
        result = numer.search(string=string)
        return result.group().strip()
    except:
        return None

In [11]:
# Test de la función 'numeracion()'
string = ['JOSE HERNANDEZ N°4859 8D','JOSE HERNANDEZ N  4859 8D','JOSE HERNANDEZ N4859 8TOD','35 N 2710',
          '141 BIS 4680','3458','GURRUCHAGA','CARMEN Nº335 5°C', 'PERÓN2019','25 DE MAYO 243 7A','552 BIS','41',
          'RUTA 8 KM 52','AV. 25 DE MAYO 322','CALLE 543 59']#,'RUTA 36 KM 41 1/2']<= Con este no funciona...

for s in string:
    print(numeracion(s))

N°4859
N  4859
N 4859
N 2710
4680
None
None
Nº335
2019
243
None
None
KM 52
322
59


### Segmentación de Unidades en la Numeración  
<a class='anchor' id='unidades'></a>
Como última función a utilizarse es la que extrae la unidad en la que se expresa la numeración. Esta se aplica sobre el resultado que se obtiene de la función _numeracion()_. Como resultado de la aplicación de esta última función muchas veces es solo un número, el cual puede hacer mención a la numeración de la calle. En forma aún más redundante se suele indicar anteponiendo "N", "N°" o "Nº". Sin embargo muchas veces indican cantidad de kilometros anteponiendo "KM", especialmente cuando se trata de rutas.

In [12]:
def unidades(string):
    try:
        string = string.strip() 
        unid = re.compile('([N°ºKM]{,2})[\s]{0,}([0-9]+)')
        result = unid.search(string=string)
        ans = result.groups()
    except:
        ans = (None,None)
    return ans

In [13]:
string = ['N°4859','N 4859','4680''Nº 335','KM 52',None]

for s in string:
    print(unidades(s))

('N°', '4859')
('N', '4859')
('', '4680')
('KM', '52')
(None, None)


### Normalizacion de direcciones
<a class='anchor' id='norm'></a>  

Esta función integra las funciones anteriores, de forma tal de poder segmentar completamente entre calles y numeracion.

In [14]:
def normalizacion(string):
    string = strip_accents(string)
    calle1,calle2,calle3 = split_streets(string)
    num1 = numeracion(calle1)
    if type(num1) is str:
        calle1 = calle1.split(num1)[0]
    num2 = numeracion(calle2)
    if type(num2) is str:
        calle2 = calle2.split(num2)[0]
    num3 = numeracion(calle3)
    if type(num3) is str:
        calle3 = calle3.split(num3)[0]
    num = [num1,num2,num3]
    for n in num:
        if n is None:
            num[num.index(n)] = ''
        else:
            pass
    num = ''.join(map(str, num))
    unid,num = unidades(num)
    return calle1,calle2,calle3,unid,num

In [19]:
string = ['BELGRANO Y ENTRE RIOS','RUTA 8 KM 52','F.ALCORTA Y PAMPA','SAN JUAN DE DIOS N°666, ENT BELGRANO Y ENTRE RIOS',
          'YATAY 541', 'CALLE 41 12 E/ 1 Y 2', 'ENTRE RIOS 334 E/ TUCUMAN Y INDEPENDENCIA']

In [20]:
for s in string:
    print(normalizacion(s))

('BELGRANO', 'ENTRE RIOS', None, None, None)
('RUTA 8 ', None, None, 'KM', '52')
('F.ALCORTA', 'PAMPA', None, None, None)
('SAN JUAN DE DIOS ', 'BELGRANO', 'ENTRE RIOS', 'N°', '666')
('YATAY ', None, None, '', '541')
('CALLE 41 ', '1', '2', '', '12')
('ENTRE RIOS ', 'TUCUMAN', 'INDEPENDENCIA', '', '334')


<span style="color: #a2cf2a; font-family: Babas; font-size: 3em;">
Tablas Auxiliares
</span>

In [21]:
# Genero el DataFrame sobre el cual se irán acumulando los datos obtenidos
georef_dataset = pd.DataFrame(columns=['original_direccion','original_barrio','original_localidad','original_municipio', 
                                       'original_departamento','original_provincia',
                                       'original_lat','original_lon',
                                       'origen_provincia','origen_departamento',
                                       'dataset_id',
                                       'original_calle1','original_calle2','original_calle3',
                                       'original_altura','original_unidades',
                                       'normalizado_calle1','normalizado_calle2','normalizado_calle3',
                                       'normalizado_altura',
                                       'normalizado_localidad','normalizado_departamento','normalizado_provincia',
                                       'normalizado_lat','normalizado_lon'])

In [22]:
# Genero el DataFrame donde se pondrá la codificación correspondiente a los datasets 
datasets = pd.DataFrame({'id':[],'dataset':[]})

## Bahra
<a class='anchor' id='bahra'></a>

In [23]:
bahra = pd.read_csv('../data/external/BAHRA_2014_version_0.5.csv')

In [24]:
bahra.head()

,cod_prov,nom_prov,cod_depto,nom_depto,cod_loc,cod_sit,cod_entidad,cod_bahra,tipo_bahra,nombre_bahra,lat_gd,long_gd,lat_gms,long_gms,fuente_ubicacion,the_geom,gid
0,2,CIUDAD DE BUENOS AIRES,0,\N,10,0,0,2000010000,LC,CIUDAD DE BUENOS AIRES,-34.613292,-58.388752,"-34° 36m 47.850""","-58° 23m 19.506""",INDEC,0101000020E61000007048ED9DC2314DC05885E757804E...,11110
1,2,CIUDAD DE BUENOS AIRES,1,COMUNA 1,10,0,1,2001010001,E,CONSTITUCION,-34.625048,-58.384387,"-34° 37m 30.172""","-58° 23m 3.794""",INDEC,0101000020E6100000E13EC39933314DC064A683910150...,80001
2,2,CIUDAD DE BUENOS AIRES,1,COMUNA 1,10,0,2,2001010002,E,MONSERRAT,-34.612688,-58.379652,"-34° 36m 45.677""","-58° 22m 46.747""",INDEC,0101000020E61000002F2D2E7098304DC02CCB25906C4E...,80002
3,2,CIUDAD DE BUENOS AIRES,1,COMUNA 1,10,0,3,2001010003,E,PUERTO MADERO,-34.609216,-58.356381,"-34° 36m 33.178""","-58° 21m 22.971""",INDEC,0101000020E610000099D1CBE39D2D4DC07137D5CBFA4D...,80003
4,2,CIUDAD DE BUENOS AIRES,1,COMUNA 1,10,0,4,2001010004,E,RETIRO,-34.588424,-58.375936,"-34° 35m 18.328""","-58° 22m 33.369""",INDEC,0101000020E61000007F4102AB1E304DC09D8A267D514B...,80004


## Código de area telefónico
<a class='anchor' id='codigo_tel'></a>

In [25]:
telefono = pd.read_excel('../data/external/Codigos de area Argentina.xls')

In [26]:
telefono.head()

,Ciudad,Provincia,Código
0,25 DE MAYO (PROV. BUENOS AIRES),BUENOS AIRES,2345
1,9 DE JULIO (PROV. BUENOS AIRES),BUENOS AIRES,2317
2,ACEBAL,SANTA FE,3469
3,ADELIA MARIA,CORDOBA,3585
4,AIMOGASTA,LA RIOJA,3827


<span style="color: #a2cf2a; font-family: Babas; font-size: 3em;">
Lista de Datasets
</span>

# data-sip.csv 
<a class="anchor" id="data-sip"></a>

In [27]:
datasets =  datasets.append({'id':0,'dataset':'data-sip.csv'}, ignore_index=True) # proviene de ENIA data/intermit

In [28]:
data = pd.read_csv('../data/raw/data-sip.csv')

In [29]:
data.head()

,Unnamed: 0,edad_mujer,id_educacion,nivel_educacion,direccion_mujer,localidad_mujer,localidad_estimada,departamento_estimado,siisa,nombre_efector,localidad_efector,departamento_efector,provincia_efector,lat_efector,lon_efector
0,0,18,5,Secundario completo,RIVADAVIA 2067,9 MDE ABRIL,9 DE ABRIL,ESTEBAN ECHEVERRIA,10020012215218,HOSPITAL GENERAL DE AGUDOS DR. CARLOS G. DURAND,CIUDAD DE BUENOS AIRES,CIUDAD DE BUENOS AIRES,CABA,-34.610111,-58.439283
1,1,18,5,Secundario completo,VELAZCO Nº 75,9 DE ABRIL,9 DE ABRIL,ESTEBAN ECHEVERRIA,10062602200434,POLICLINICO SOFIA TERRERO DE SANTA MARINA,MONTE GRANDE,ESTEBAN ECHEVERRIA,Buenos Aires,-34.813971,-58.459338
2,2,18,5,Secundario completo,SANTA TERESITA 1847,9 DE ABRIL,9 DE ABRIL,ESTEBAN ECHEVERRIA,10062602200434,POLICLINICO SOFIA TERRERO DE SANTA MARINA,MONTE GRANDE,ESTEBAN ECHEVERRIA,Buenos Aires,-34.813971,-58.459338
3,3,18,5,Secundario completo,SANTA INES 4658,9 DE ABRIL,9 DE ABRIL,ESTEBAN ECHEVERRIA,10062602200434,POLICLINICO SOFIA TERRERO DE SANTA MARINA,MONTE GRANDE,ESTEBAN ECHEVERRIA,Buenos Aires,-34.813971,-58.459338
4,4,18,5,Secundario completo,SAN ASNTONIO Nº 1269,9 DE ABRIL,9 DE ABRIL,ESTEBAN ECHEVERRIA,10062602200434,POLICLINICO SOFIA TERRERO DE SANTA MARINA,MONTE GRANDE,ESTEBAN ECHEVERRIA,Buenos Aires,-34.813971,-58.459338


In [30]:
data.columns

Index(['Unnamed: 0', 'edad_mujer', 'id_educacion', 'nivel_educacion',
       'direccion_mujer', 'localidad_mujer', 'localidad_estimada',
       'departamento_estimado', 'siisa', 'nombre_efector', 'localidad_efector',
       'departamento_efector', 'provincia_efector', 'lat_efector',
       'lon_efector'],
      dtype='object')

In [31]:
data = data[['direccion_mujer', 'localidad_mujer', 'localidad_estimada', 'departamento_estimado', 
             'departamento_efector', 'provincia_efector']]

In [32]:
data.rename(columns={'direccion_mujer':'original_direccion', 'localidad_mujer':'original_localidad', 
                     'localidad_estimada':'normalizado_localidad', 'departamento_estimado':'normalizado_departamento', 
                     'departamento_efector':'origen_departamento', 'provincia_efector':'origen_provincia'},
            inplace=True)

In [33]:
data[['original_calle1','original_calle2','original_calle3','original_unidades','original_altura',]] = data['original_direccion'].dropna().apply(lambda x: normalizacion(x)).apply(pd.Series)

In [34]:
data[['original_direccion','original_calle1','original_calle2','original_calle3','original_unidades','original_altura']].head(20)

,original_direccion,original_calle1,original_calle2,original_calle3,original_unidades,original_altura
0,RIVADAVIA 2067,RIVADAVIA,None,None,,2067
1,VELAZCO Nº 75,VELAZCO,None,None,Nº,75
2,SANTA TERESITA 1847,SANTA TERESITA,None,None,,1847
3,SANTA INES 4658,SANTA INES,None,None,,4658
4,SAN ASNTONIO Nº 1269,SAN ASNTONIO,None,None,Nº,1269
5,PEDRO CONI Nº 3597,PEDRO CONI,None,None,Nº,3597
6,MALVINAS Nº 4018,MALVINAS,None,None,Nº,4018
7,LANIN Nº 449,LANIN,None,None,Nº,449
8,LA PAZ 1095,LA PAZ,None,None,,1095
9,GAREADOR Nº 626,GAREADOR,None,None,Nº,626


In [35]:
# Agrego codigo del archivo del dataset para todos los registros
data['dataset_id'] = 0

In [36]:
georef_dataset = georef_dataset.append(data)

/home/ezequiel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [37]:
georef_dataset

,dataset_id,normalizado_altura,normalizado_calle1,normalizado_calle2,normalizado_calle3,normalizado_departamento,normalizado_lat,normalizado_localidad,normalizado_lon,normalizado_provincia,...,original_calle2,original_calle3,original_departamento,original_direccion,original_lat,original_localidad,original_lon,original_municipio,original_provincia,original_unidades
0,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,RIVADAVIA 2067,NaN,9 MDE ABRIL,NaN,NaN,NaN,
1,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,VELAZCO Nº 75,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº
2,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,SANTA TERESITA 1847,NaN,9 DE ABRIL,NaN,NaN,NaN,
3,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,SANTA INES 4658,NaN,9 DE ABRIL,NaN,NaN,NaN,
4,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,SAN ASNTONIO Nº 1269,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº
5,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,PEDRO CONI Nº 3597,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº
6,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,MALVINAS Nº 4018,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº
7,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,LANIN Nº 449,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº
8,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,LA PAZ 1095,NaN,9 DE ABRIL,NaN,NaN,NaN,
9,0,NaN,NaN,NaN,NaN,ESTEBAN ECHEVERRIA,NaN,9 DE ABRIL,NaN,NaN,...,None,None,NaN,GAREADOR Nº 626,NaN,9 DE ABRIL,NaN,NaN,NaN,Nº


<a class="anchor" id="PARANA"></a>
# PARANA - Consolidado Dia 1 y 2.xlsx

In [38]:
datasets = datasets.append({'id':1,'dataset':'PARANA - Consolidado Dia 1 y 2.xlsx'}, ignore_index=True) # proviene de Empleo Joven

In [39]:
data =  pd.read_excel('../data/raw/PARANA - Consolidado Dia 1 y 2.xlsx')

In [41]:
data.head()

,Nombre,Apellido,DNI,CUIL,Sexo,FechaNac,Correo,prefijo,Telefono,prefijoCel,...,IdLocalidad,IdNivelEducativo,AsistioEsc,IdTitulo,EstadoEsc,MasInfo,Trabaja,Redes,Nombre red,Usuario red
0,Anibal,---,32619328,NaN,1.0,1958-05-03 00:00:00,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARNDRES,ABADIE,25925049,20259250499,1.0,1977-11-04 00:00:00,NaN,NaN,NaN,0.0,...,2453.0,TER,Si,103.0,Completo,NaN,Si,Si,FACEBOOK,NaN
2,ARNDRES,ABADIE,25925049,20259250499,1.0,1977-11-04 00:00:00,NaN,NaN,NaN,0.0,...,2453.0,TER,Si,103.0,Completo,NaN,Si,Si,FACEBOOK,NaN
3,MELISA,PEREZ,43114078,NaN,2.0,2000-11-28 00:00:00,PEREZMELISA28@GMAIL.COM,NaN,NaN,343.0,...,NaN,SEC,Si,NaN,En curso,NaN,No,Si,NaN,NaN
4,LUCENA,ABASTO,35709116,NaN,2.0,1991-03-05 00:00:00,NaN,NaN,NaN,0.0,...,2453.0,UNI,Si,NaN,En curso,NaN,No,Si,NaN,NaN


In [42]:
data.columns

Index(['Nombre', 'Apellido', 'DNI', 'CUIL', 'Sexo', 'FechaNac', 'Correo',
       'prefijo', 'Telefono', 'prefijoCel', 'celular', 'Direccion', 'IdPais',
       'Provincia_MTSS', 'IDMuni', 'IdDepto', 'IdLocalidad',
       'IdNivelEducativo', 'AsistioEsc', 'IdTitulo', 'EstadoEsc', 'MasInfo',
       'Trabaja', 'Redes', 'Nombre red', 'Usuario red'],
      dtype='object')

In [43]:
data = data[['Direccion', 'Provincia_MTSS', 'IDMuni', 'IdDepto', 'IdLocalidad','prefijo','prefijoCel']]
data.head()

,Direccion,Provincia_MTSS,IDMuni,IdDepto,IdLocalidad,prefijo,prefijoCel
0,NaN,30.0,872.0,263.0,NaN,NaN,0.0
1,JUAN JOSÃˆ PASOS 67,30.0,872.0,263.0,2453.0,NaN,0.0
2,JUAN JOSÃˆ PASOS 67,30.0,872.0,263.0,2453.0,NaN,0.0
3,ROQUES SAENS PEÃ‘A 1355,30.0,872.0,263.0,NaN,NaN,343.0
4,BARRIO COVIALER,30.0,872.0,263.0,2453.0,NaN,0.0


In [44]:
data['prefijo'].value_counts()

343.0    55
3.0       4
Name: prefijo, dtype: int64

In [45]:
data['prefijo'].isnull().sum()

2229

In [46]:
data['prefijoCel'].value_counts()

343.0        1137
0.0           951
3435.0         20
3434.0         12
3430.0         10
3437.0         10
345.0           8
342.0           8
11.0            7
3454.0          6
34.0            4
3445.0          4
434.0           4
351.0           4
3438.0          3
3447.0          3
34345.0         2
2804.0          2
3446.0          2
3498.0          2
3455.0          2
3704.0          2
43.0            2
3431167.0       2
3462.0          1
2223.0          1
3751.0          1
3492.0          1
3442.0          1
3401.0          1
298.0           1
3.0             1
3496.0          1
Name: prefijoCel, dtype: int64

In [47]:
data['prefijoCel'].isnull().sum()

72

In [48]:
telefono.loc[telefono['Código']==343]

,Ciudad,Provincia,Código
182,PARANA,ENTRE RIOS,343


In [35]:
data = data.merge(bahra[['cod_prov','nom_prov']].drop_duplicates(),how='left',left_on=['Provincia_MTSS'],right_on='cod_prov')

In [36]:
# El codigo de departamentos no es bahra.....
#data = data.merge(bahra[['cod_depto','nom_depto']].drop_duplicates(),how='left',left_on=['IdDepto'],right_on='cod_depto')

In [37]:
data = data.merge(telefono[['Código','Ciudad']].drop_duplicates(),how='left',left_on=['prefijoCel'],right_on='Código')

In [38]:
data.head(20)

,Direccion,Provincia_MTSS,IDMuni,IdDepto,IdLocalidad,prefijo,prefijoCel,cod_prov,nom_prov,Código,Ciudad
0,NaN,30.0,872.0,263.0,NaN,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
1,JUAN JOSÃˆ PASOS 67,30.0,872.0,263.0,2453.0,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
2,JUAN JOSÃˆ PASOS 67,30.0,872.0,263.0,2453.0,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
3,ROQUES SAENS PEÃ‘A 1355,30.0,872.0,263.0,NaN,NaN,343.0,30.0,ENTRE RIOS,343.0,PARANA
4,BARRIO COVIALER,30.0,872.0,263.0,2453.0,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
5,SARAVI 427,30.0,872.0,263.0,2453.0,NaN,343.0,30.0,ENTRE RIOS,343.0,PARANA
6,julio roca 113,30.0,872.0,263.0,2453.0,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
7,gobernador mihura 1381,30.0,872.0,263.0,2453.0,NaN,0.0,30.0,ENTRE RIOS,NaN,NaN
8,guillani 99,30.0,870.0,262.0,2420.0,NaN,345.0,30.0,ENTRE RIOS,345.0,CONCORDIA
9,guillani 99,30.0,870.0,262.0,2420.0,NaN,345.0,30.0,ENTRE RIOS,345.0,CONCORDIA


In [39]:
# Tomar la caracteristica del cel para ubicar la localidad/depto me obliga a tomar tambien provincia....
# y de ahi tengo un conflicto con los datos relevados, no muy grande ya que creo mas en el dato del celu
data.groupby(['nom_prov','IdDepto','Ciudad']).size()

nom_prov      IdDepto  Ciudad                       
BUENOS AIRES  3.0      PARANA                              2
              6.0      PARANA                              2
              13.0     CAPITAL FEDERAL y alrededores       1
              15.0     PARANA                              2
              17.0     CORONEL BRANDSEN                    1
              20.0     CAPITAL FEDERAL y alrededores       1
                       PARANA                              1
              55.0     PARANA                              1
              68.0     PARANA                              2
              104.0    CAPITAL FEDERAL y alrededores       1
              139.0    PARANA                              1
CORRIENTES    186.0    PARANA                              2
              209.0    PARANA                              1
ENTRE RIOS    2.0      PARANA                              1
              252.0    COLON (PROV.  ENTRE RIOS)           2
              253.0    CONCORDIA

In [40]:
data.columns

Index(['Direccion', 'Provincia_MTSS', 'IDMuni', 'IdDepto', 'IdLocalidad',
       'prefijo', 'prefijoCel', 'cod_prov', 'nom_prov', 'Código', 'Ciudad'],
      dtype='object')

In [41]:
data = data[['Direccion', 'nom_prov','Ciudad']]

In [42]:
data['origen_provincia'] = 'ENTRE RIOS'
data['origen_departamento'] = 'PARANA'

In [43]:
data.rename(columns={'Direccion':'original_direccion', 'nom_prov':'original_provincia', 'Ciudad':'original_localidad'},
            inplace=True)

In [44]:
# Agrego codigo del archivo del dataset para todos los registros
data['dataset_id'] = 1

In [45]:
data.head() # Hay que limpiar el texto de las direcciones

,original_direccion,original_provincia,original_localidad,origen_provincia,origen_departamento,dataset_id
0,NaN,ENTRE RIOS,NaN,ENTRE RIOS,PARANA,1
1,JUAN JOSÃˆ PASOS 67,ENTRE RIOS,NaN,ENTRE RIOS,PARANA,1
2,JUAN JOSÃˆ PASOS 67,ENTRE RIOS,NaN,ENTRE RIOS,PARANA,1
3,ROQUES SAENS PEÃ‘A 1355,ENTRE RIOS,PARANA,ENTRE RIOS,PARANA,1
4,BARRIO COVIALER,ENTRE RIOS,NaN,ENTRE RIOS,PARANA,1


In [46]:
#georef_dataset = georef_dataset.append(data)

<a class='anchor' id='RIO IV'></a>
# RIO IV - Consolidado Dia 1 y 2.xlsx

In [47]:
datasets = datasets.append({'id':2,'dataset':'RIO IV - Consolidado Dia 1 y 2.xlsx'}, ignore_index=True) # proviene de Empleo Joven

In [48]:
data = pd.read_excel('../data/raw/RIO IV - Consolidado Dia 1 y 2.xlsx')

In [49]:
data.head()

,Nombre,Apellido,DNI,CUIL,Sexo,FechaNac,Correo,prefijo,Telefono,prefijoCel,celular,Direccion,IdNivelEducativo,AsistioEsc,EstadoEsc,Trabaja,Redes,Nombrered,Usuariored
0,Carolina,Villegas,39473362,NaN,2.0,1996-07-18 00:00:00,NaN,NaN,NaN,0.0,NaN,Pje. J Curchod 857,UNI,Si,En curso,No,Si,NaN,NaN
1,CHRISTIAN,FERREYRA,41575139,20415751398,1.0,1999-09-01 00:00:00,NaN,NaN,NaN,NaN,NaN,HUARPES 2367,SEC,Si,Completo,No,No,NaN,NaN
2,Tatiana,gonzalez,35915494,27359154947,2.0,1991-09-28 00:00:00,NaN,NaN,NaN,0.0,NaN,Union de los argentinos 408,SEC,Si,Completo,No,No,NaN,NaN
3,Pamela,Coria,41595393,NaN,2.0,1999-01-19 00:00:00,NaN,NaN,NaN,0.0,NaN,Adelia mar¡a 969,SEC,Si,En curso,No,Si,NaN,NaN
4,Marisa,Zavala,38730478,NaN,2.0,1995-01-05 00:00:00,NaN,NaN,NaN,0.0,NaN,Bethoven 50,TER,Si,En curso,No,No,NaN,NaN


In [50]:
data['prefijo'].value_counts() # algo se podria hacer con la caracteristica telefonica

358.0     89
3571.0     2
3534.0     2
3585.0     2
3.0        2
Name: prefijo, dtype: int64

In [51]:
data['prefijoCel'].value_counts()

0.0        2285
358.0      2053
3583.0       35
3582.0       30
3584.0       27
3468.0       23
351.0        23
3585.0       21
3571.0       19
353.0        18
2657.0       12
3385.0       12
2664.0       10
2302.0        9
11.0          9
388.0         7
3537.0        7
3580.0        6
2622.0        5
2336.0        4
385.0         4
3548.0        3
368.0         3
266.0         3
264.0         3
3544.0        3
354.0         3
2954.0        3
2945.0        2
2342.0        2
           ... 
3546.0        2
3888.0        2
3382.0        2
3444.0        2
2335.0        2
297.0         2
2966.0        2
2625.0        1
387.0         1
22664.0       1
58.0          1
3587.0        1
299.0         1
341.0         1
302.0         1
381.0         1
3535.0        1
295.0         1
3471.0        1
3541.0        1
2901.0        1
3567.0        1
3558.0        1
357.0         1
2994.0        1
3358.0        1
261.0         1
359.0         1
3886.0        1
335.0         1
Name: prefijoCel, Length

In [52]:
data.loc[data['prefijoCel']==0]

,Nombre,Apellido,DNI,CUIL,Sexo,FechaNac,Correo,prefijo,Telefono,prefijoCel,celular,Direccion,IdNivelEducativo,AsistioEsc,EstadoEsc,Trabaja,Redes,Nombrered,Usuariored
0,Carolina,Villegas,39473362,NaN,2.0,1996-07-18 00:00:00,NaN,NaN,NaN,0.0,NaN,Pje. J Curchod 857,UNI,Si,En curso,No,Si,NaN,NaN
2,Tatiana,gonzalez,35915494,27359154947,2.0,1991-09-28 00:00:00,NaN,NaN,NaN,0.0,NaN,Union de los argentinos 408,SEC,Si,Completo,No,No,NaN,NaN
3,Pamela,Coria,41595393,NaN,2.0,1999-01-19 00:00:00,NaN,NaN,NaN,0.0,NaN,Adelia mar¡a 969,SEC,Si,En curso,No,Si,NaN,NaN
4,Marisa,Zavala,38730478,NaN,2.0,1995-01-05 00:00:00,NaN,NaN,NaN,0.0,NaN,Bethoven 50,TER,Si,En curso,No,No,NaN,NaN
5,Mayra,lopez,41887953,NaN,2.0,1999-05-29 00:00:00,NaN,NaN,NaN,0.0,NaN,Paul grossac 980,SEC,Si,En curso,No,No,NaN,NaN
6,Soraya,Virga,35544704,NaN,2.0,1991-12-03 00:00:00,NaN,NaN,NaN,0.0,NaN,Parana 16,UNI,Si,En curso,No,No,NaN,NaN
7,Natalia,Montes,36970280,NaN,2.0,1992-02-12 00:00:00,NaN,NaN,NaN,0.0,NaN,Maipu 2928,TER,Si,En curso,No,No,NaN,NaN
8,Maximiliano,Quiroga,42697467,NaN,1.0,2000-06-12 00:00:00,NaN,NaN,NaN,0.0,NaN,Nahuel Huapi 1827,SEC,Si,En curso,Si,No,NaN,NaN
9,Karen,Rodriguez,39968271,NaN,2.0,1997-05-11 00:00:00,NaN,NaN,NaN,0.0,NaN,Alfredo terzaga 820,SEC,Si,En curso,No,Si,Facebook,NaN
10,Gabriel,lopez,42638437,NaN,1.0,2000-04-29 00:00:00,NaN,NaN,NaN,0.0,NaN,Tomas Suarez 767,SEC,Si,En curso,No,No,NaN,NaN


In [53]:
data['prefijoCel'].isnull().sum()

53

In [54]:
telefono.loc[telefono['Código']==358]

,Ciudad,Provincia,Código
208,RIO CUARTO,CORDOBA,358


In [55]:
telefono.loc[telefono['Código']==3582]

,Ciudad,Provincia,Código
227,SAMPACHO,CORDOBA,3582


In [56]:
data = data.merge(telefono[['Código','Ciudad']].drop_duplicates(),how='left',left_on=['prefijoCel'],right_on='Código')

In [57]:
# Agrego codigo del archivo del dataset para todos los registros
data['dataset_id'] = 2

In [58]:
#georef_dataset = georef_dataset.append(data)

<a class='anchor' id='FORTALECIMIENTO'></a>
# FORTALECIMIENTO REPI.xlsx

In [59]:
data = pd.read_excel('../data/raw/FORTALECIMIENTO REPI.xlsx',skiprows=[0])

In [60]:
data.head()

,ORGANISMO,DOMICILIO,LOCALIDAD,PROVINCIA,NUEVO / A FORTALECER,DOMICILIO.1,LOCALIDAD.1,PROVINCIA.1
0,EL MINISTERIO DE DESARROLLO SOCIAL DEL GOBIERN...,Calle 55 N° 570,La Plata,Buenos Aires,el fortalecimiento,Av. Del Libertador N° 7914 esq. Pasaje 1,Loma Hermosa,Buenos Aires
1,LA ASOCIACIÓN CIVIL PEQUEÑOS PASOS,Perú 168 sdo. 11,Acassuso,Buenos Aires,el fortalecimiento,Solís 6963,Loma Hermosa,Buenos Aires
2,LA ASOCIACIÓN CIVIL U.N.I.D.O.S. (UNIÓN DE OR...,Curumalal 8110,Virrey del Pino,Buenos Aires,el fortalecimiento,Beethoven 2823,Rafael Castillo,Buenos Aires
3,LA ASOCIACIÓN CIVIL U.N.I.D.O.S. (UNIÓN DE OR...,Curumalal 8110,Virrey del Pino,Buenos Aires,el fortalecimiento,Tornquist 6576,Isidro Casanova,Buenos Aires
4,LA ASOCIACIÓN CIVIL U.N.I.D.O.S. (UNIÓN DE OR...,Curumalal 8110,Virrey del Pino,Buenos Aires,el fortalecimiento,Puerto Argerntino 5267,Gonzalez Catán,Buenos Aires


In [61]:
data.columns

Index(['ORGANISMO', 'DOMICILIO', 'LOCALIDAD', 'PROVINCIA',
       'NUEVO / A FORTALECER ', 'DOMICILIO.1', 'LOCALIDAD.1', 'PROVINCIA.1'],
      dtype='object')

In [62]:
aux = data[['DOMICILIO.1', 'LOCALIDAD.1', 'PROVINCIA.1']]
aux = aux.rename(columns={'DOMICILIO.1':'DOMICILIO', 'LOCALIDAD.1':'LOCALIDAD', 'PROVINCIA.1':'PROVINCIA'})
data = data[['DOMICILIO', 'LOCALIDAD', 'PROVINCIA']].append(aux)

In [63]:
data.rename(columns={'DOMICILIO':'original_direccion', 'LOCALIDAD':'original_localidad', 
                     'PROVINCIA':'original_provincia'},
           inplace=True)

In [64]:
# Agrego codigo del archivo del dataset para todos los registros
data['dataset_id'] = 3

In [65]:
data.head()

,original_direccion,original_localidad,original_provincia,dataset_id
0,Calle 55 N° 570,La Plata,Buenos Aires,3
1,Perú 168 sdo. 11,Acassuso,Buenos Aires,3
2,Curumalal 8110,Virrey del Pino,Buenos Aires,3
3,Curumalal 8110,Virrey del Pino,Buenos Aires,3
4,Curumalal 8110,Virrey del Pino,Buenos Aires,3


In [66]:
#georef_dataset = georef_dataset.append(data)

<a class='anchor' id='500 casos'></a>
# 500_CASOS_A_NORMALIZAR.xls

In [67]:
data = pd.read_excel('../data/raw/500_CASOS_A_NORMALIZAR.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [68]:
data.head(10) # la calle es una mezcla de cosas, especialmente en La Plata

,cod_provincia_detalle,depto_norm,loca_norm,calle_hij,nro_hij
0,BUENOS AIRES,MORENO,CUARTEL V,DON BOSCO,04866
1,BUENOS AIRES,ALMIRANTE BROWN/QUILMES,SAN FRANCISCO SOLANO,HUMBERTO PRIMO,6398
2,BUENOS AIRES,ARRECIFES,ARRECIFES,JULIAN CENTELLA CASA 36,00860
3,BUENOS AIRES,ENSENADA,ENSENADA,QUINTANA Y BOLIVIA BIS,00004
4,BUENOS AIRES,GENERAL BELGRANO,GENERAL BELGRANO,ALVEAR,00555
5,BUENOS AIRES,AVELLANEDA,DOCK SUD,MANUEL ESTEVEZ,01110
6,BUENOS AIRES,LA PLATA,LA PLATA,141 E/78 Y 78BIS,00070
7,BUENOS AIRES,NECOCHEA,NECOCHEA,67 BIS,04096
8,BUENOS AIRES,ALMIRANTE BROWN,LONGCHAMPS,MTRO FRENCH,01712
9,BUENOS AIRES,LA MATANZA,VILLA EDUARDO MADERO,UGARTE,01186


In [69]:
aux = data.loc[data['calle_hij'].str.contains('E/')]['calle_hij'].str.split('E/',expand=True)
data['original_calle1'] = aux[0]
data['original_calle2'] = aux[1].str.split(' Y ',expand=True)[0]
data['original_calle3'] = aux[1].str.split(' Y ',expand=True)[1]

In [70]:
data.loc[data['calle_hij'].str.contains('E/')]

,cod_provincia_detalle,depto_norm,loca_norm,calle_hij,nro_hij,original_calle1,original_calle2,original_calle3
6,BUENOS AIRES,LA PLATA,LA PLATA,141 E/78 Y 78BIS,00070,141,78,78BIS
11,BUENOS AIRES,COLON,COLON,58 BIS E/ 22 Y 23,1066,58 BIS,22,23
32,BUENOS AIRES,LA PLATA,LA PLATA,522 E/ 135 Y 136,04046,522,135,136
59,BUENOS AIRES,LA PLATA,LA PLATA,122 BIS E/ 609 Y 610,NaN,122 BIS,609,610
130,BUENOS AIRES,LA PLATA,JOSE HERNANDEZ,131 BIS E/517 Y 518,01410,131 BIS,517,518
203,BUENOS AIRES,BERISSO,BERISSO,2 E/ 168 Y 169,04661,2,168,169
208,BUENOS AIRES,FLORENCIO VARELA,VILLA VATTEONE,RIO CARAGUATA E/ POMPEYA Y LOURDES,02471,RIO CARAGUATA,POMPEYA,LOURDES
248,BUENOS AIRES,LA PLATA,JOSE MELCHOR ROMERO,185E/527Y528,99999,185,527Y528,None
290,BUENOS AIRES,LA PLATA,LA PLATA,CALLE 93 E/ 2 Y 3,09999,CALLE 93,2,3
297,BUENOS AIRES,ENSENADA,ENSENADA,14 BIS E/ 9 Y 11,99999,14 BIS,9,11


In [71]:
data.columns

Index(['cod_provincia_detalle', 'depto_norm', 'loca_norm', 'calle_hij',
       'nro_hij', 'original_calle1', 'original_calle2', 'original_calle3'],
      dtype='object')

In [72]:
data.rename(columns={'cod_provincia_detalle':'original_provincia', 'depto_norm':'original_departamento', 
             'loca_norm':'original_localidad', 'calle_hij':'original_calle1','nro_hij':'original_altura'},
            inplace=True)

In [73]:
#georef_dataset = georef_dataset.append(data)

<a class='anchor' id='500 casos geo'></a>
# 500_CASOS_GEOCONFIRMADOS.xls

In [74]:
data = pd.read_excel('../data/raw/500_CASOS_GEOCONFIRMADOS.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [75]:
data.loc[data['depto_norm']=='LA PLATA'].head(20)

,cod_provincia_detalle,depto_norm,loca_norm,calle_hij,nro_hij,calle_norm,nro_norm,dire_norm,lat,lng
16,BUENOS AIRES,LA PLATA,LA PLATA,159,00009,CALLE 159,NaN,CALLE 159 9,-34.951349,-58.020443
42,BUENOS AIRES,LA PLATA,LOS HORNOS,CALLE 139,1391,CALLE 139,NaN,CALLE 139 1391,-34.952033,-57.973904
71,BUENOS AIRES,LA PLATA,LA PLATA,509 E/ 30 Y 31,03542,CALLE 509,NaN,CALLE 509 3542,-34.903132,-58.018737
102,BUENOS AIRES,LA PLATA,LA PLATA,5 Y 91,99999,CALLE 5,NaN,CALLE 5 & CALLE 91,-34.900299,-57.964887
118,BUENOS AIRES,LA PLATA,LA PLATA,78 BIS E 21 Y 22,01326,CALLE 78,NaN,CALLE 78 1326,-34.949495,-57.936009
173,BUENOS AIRES,LA PLATA,LA PLATA,51,01164,CALLE 51,NaN,CALLE 51 1164,-34.926007,-57.960493
187,BUENOS AIRES,LA PLATA,LOS HORNOS,145 E/ 60 Y 61,1389,CALLE 145,NaN,CALLE 145 1389,-34.957635,-57.980232
259,BUENOS AIRES,LA PLATA,LA PLATA,512,00926,CALLE 512,NaN,CALLE 512 926,-34.881224,-57.988686
302,BUENOS AIRES,LA PLATA,LISANDRO OLMOS,45 E/ 201 Y 202,5322,CALLE 45,NaN,CALLE 45 5322,-35.000494,-58.051519
396,BUENOS AIRES,LA PLATA,LA PLATA,DIAG 74,00039,CALLE 74,NaN,CALLE 74 39,-34.890128,-57.956749


In [76]:
aux = data.loc[data['calle_hij'].str.contains('E/')]['calle_hij'].str.split('E/',expand=True)
data['original_calle1'] = aux[0]
data['original_calle2'] = aux[1].str.split(' Y ',expand=True)[0]
data['original_calle3'] = aux[1].str.split(' Y ',expand=True)[1]

In [77]:
data.loc[data['calle_hij'].str.contains('E/')]

,cod_provincia_detalle,depto_norm,loca_norm,calle_hij,nro_hij,calle_norm,nro_norm,dire_norm,lat,lng,original_calle1,original_calle2,original_calle3
71,BUENOS AIRES,LA PLATA,LA PLATA,509 E/ 30 Y 31,03542,CALLE 509,NaN,CALLE 509 3542,-34.903132,-58.018737,509,30,31
187,BUENOS AIRES,LA PLATA,LOS HORNOS,145 E/ 60 Y 61,1389,CALLE 145,NaN,CALLE 145 1389,-34.957635,-57.980232,145,60,61
210,BUENOS AIRES,GENERAL RODRIGUEZ,GENERAL RODRIGUEZ,CUBA E/ MERCEDES Y FLORI,00000,NaN,NaN,CUBA & MERCEDES,-34.626354,-58.891243,CUBA,MERCEDES,FLORI
302,BUENOS AIRES,LA PLATA,LISANDRO OLMOS,45 E/ 201 Y 202,5322,CALLE 45,NaN,CALLE 45 5322,-35.000494,-58.051519,45,201,202
398,BUENOS AIRES,LA PLATA,LISANDRO OLMOS,203 E/ 59 Y 60,S/N,CALLE 203,NaN,CALLE 203 & CALLE 59,-34.990342,-58.068743,203,59,60
451,BUENOS AIRES,LA PLATA,CITY BELL,467 E/ 142 Y 148,00237,CALLE 467,NaN,CALLE 467 237,-34.866626,-58.047112,467,142,148
482,BUENOS AIRES,LA PLATA,CITY BELL,28BIS E/ 458 Y 459,1391,CALLE 28 BIS,NaN,CALLE 28 BIS 1391,-34.885739,-58.066178,28BIS,458,459


In [78]:
aux = data.loc[data['calle_hij'].str.contains(' E ')]['calle_hij'].str.split(' E ',expand=True)
data['original_calle1'] = aux[0]
data['original_calle2'] = aux[1].str.split(' Y ',expand=True)[0]
data['original_calle3'] = aux[1].str.split(' Y ',expand=True)[1]

In [79]:
data.loc[data['calle_hij'].str.contains(' E ')]

,cod_provincia_detalle,depto_norm,loca_norm,calle_hij,nro_hij,calle_norm,nro_norm,dire_norm,lat,lng,original_calle1,original_calle2,original_calle3
38,BUENOS AIRES,LOMAS DE ZAMORA,LOMAS DE ZAMORA,MILLAN E CALLE 6 Y LA 10,00394,MILLAN,394.0,MILLAN 394,-34.728317,-58.430042,MILLAN,CALLE 6,LA 10
118,BUENOS AIRES,LA PLATA,LA PLATA,78 BIS E 21 Y 22,01326,CALLE 78,NaN,CALLE 78 1326,-34.949495,-57.936009,78 BIS,21,22
400,BUENOS AIRES,QUILMES,QUILMES,AV GUILLERMO E HUDSON,01563,GUILLERMO HUDSON,1563.0,GUILLERMO HUDSON 1563,-34.753963,-58.326977,AV GUILLERMO,HUDSON,None


In [80]:
#georef_dataset = georef_dataset.append(data)

<a class='anchor' id='CUS'></a>
# CUS-medicamentos.xlsx

In [81]:
data = pd.read_excel('../data/raw/CUS-medicamentos.xlsx')

In [82]:
data.head()

,priorizada_por_el_plan,incluidos_en_el_consolidado,nroefector,establecimiento,domicilio,localidad,municipio,departamento,provincia,region,...,subcategoriadescripcion,habilitado,fechahabilitacion,nro_siisa,estado,dependiente,dependede,domicilio.1,lat,lon
0,1,1,´07723,PS EL PUESTITO,PPAL S/N,EL PUESTITO,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,NOA,...,CAPS,SI,2004-08-24,´50861752192272,Activo,SI,´07116,"PPAL S/N, EL PUESTITO, SAN MARTIN, SANTIAGO DE...",-43.250880,-65.302580
1,1,1,´09293,PS ATOJ POZO,PPAL S/N,ATOJ POZO,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,NOA,...,CAPS,SI,2004-08-24,´50861752192217,Activo,SI,´07116,"PPAL S/N, ATOJ POZO, SAN MARTIN, SANTIAGO DEL ...",-38.837440,-68.045540
2,1,1,´10892,PS MAJADA,PPAL S/N,MAJADA,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,NOA,...,CAPS,SI,2004-08-24,´50861752192338,Activo,SI,´07116,"PPAL S/N, MAJADA, SAN MARTIN, SANTIAGO DEL EST...",-36.711850,-56.681650
3,1,1,´04037,PS CHELCOS,PPAL S/N,CHELCOS,ROSARIO VERA PEÑALOZA,ROSARIO VERA PEÑALOZA,LA RIOJA,CUYO,...,CAPS,SI,2004-12-01,´50461122155081,Activo,SI,´04035,"PPAL S/N, CHELCOS, ROSARIO VERA PEÑALOZA, LA R...",-31.271091,67.025702
4,1,1,´10914,PS SANTA ROSA (MORENO),PPAL S/N,SANTA ROSA,MORENO,MORENO,SANTIAGO DEL ESTERO,NOA,...,CAPS,SI,2004-08-24,´50861192192725,Activo,SI,´07082,"PPAL S/N, SANTA ROSA, MORENO, SANTIAGO DEL EST...",-36.612070,-64.292460


In [83]:
data.columns

Index(['priorizada_por_el_plan', 'incluidos_en_el_consolidado', 'nroefector',
       'establecimiento', 'domicilio', 'localidad', 'municipio',
       'departamento', 'provincia', 'region', 'datosadicionales', 'telefono',
       'horarioentrega', 'horarioatencion', 'categoriacodigo',
       'categoriadescripcion', 'subcategoriacodigo', 'subcategoriadescripcion',
       'habilitado', 'fechahabilitacion', 'nro_siisa', 'estado', 'dependiente',
       'dependede', 'domicilio.1', 'lat', 'lon'],
      dtype='object')

In [84]:
data = data[['domicilio', 'localidad', 'municipio','departamento', 'provincia', 'lat', 'lon']]

In [85]:
data.head()

,domicilio,localidad,municipio,departamento,provincia,lat,lon
0,PPAL S/N,EL PUESTITO,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,-43.250880,-65.302580
1,PPAL S/N,ATOJ POZO,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,-38.837440,-68.045540
2,PPAL S/N,MAJADA,SAN MARTIN,SAN MARTIN,SANTIAGO DEL ESTERO,-36.711850,-56.681650
3,PPAL S/N,CHELCOS,ROSARIO VERA PEÑALOZA,ROSARIO VERA PEÑALOZA,LA RIOJA,-31.271091,67.025702
4,PPAL S/N,SANTA ROSA,MORENO,MORENO,SANTIAGO DEL ESTERO,-36.612070,-64.292460


<a class='anchor' id='CPAP'></a>
# refes_centros_publicos_de_atencion_primaria_22-06-2018.xlsx

In [86]:
data = pd.read_excel('../data/raw/refes_centros_publicos_de_atencion_primaria_22-06-2018.xlsx')

In [169]:
data.head()

,CODIGO,GLN,NOMBRE,CODIGO_TIPOLOGIA,TIPOLOGIA,CODIGO_CATEGORIA_TIPOLOGIA,CATEGORIA_TIPOLOGIA,CODIGO_DEPENDENCIA,DEPENDENCIA,ORIGEN_FINANCIAMIENTO,...,LOCALIDAD,CP,DOMICILIO,TE1,TE2,TE3,TE4,LATITUD,LONGITUD,ESTADO_GEO
0,50380842150137,NaN,CAPS EL TORO (17),ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Sin atención médica en forma periódica (menor ...,21,Provincial,Público,...,EL TORO,4411,AVENIDA LAVALLE ESQUINA 24 DE OCTUBRE,NaN,NaN,NaN,NaN,-23.208534,-66.827087,NaN
1,50380632150075,NaN,CAPS BARRO NEGRO (9) (EX LOTE ROSARIO DE RIO G...,ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Con atención médica general por lo menos 3 día...,21,Provincial,Público,...,LA MENDIETA,4522,CARLOS GARDEL S/N° - LOTE BARRO NEGRO,NaN,NaN,NaN,NaN,-24.308135,-64.931750,NaN
2,50260212329179,NaN,KIN SPORT,ESSIDT,Establecimiento de salud sin internación de di...,ESSDIT,Con atención médica diaria y con especialidade...,23,Privado,Privado,...,COMODORO RIVADAVIA,3240,Avenida Rivadavia 633,0297-4475310,NaN,NaN,NaN,-45.862127,-67.483555,NaN
3,51260352329290,NaN,LABORATORIO ANALISIS CLINICOS GEROSA,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,25 de Mayo 498,2945452050,294515503809,NaN,NaN,-42.915051,-71.319707,NaN
4,51260352329289,NaN,LABORATORIO ANALISIS CLINICOS,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,Ameghino 1033,02945-452155,NaN,NaN,NaN,-42.915771,-71.318034,NaN


<a class='anchor' id='ESP'></a>
# refes_establecimientos_de_salud_privados_22-06-2018

In [87]:
data = pd.read_excel('../data/raw/refes_establecimientos_de_salud_privados_22-06-2018.xlsx')

In [170]:
data.head()

,CODIGO,GLN,NOMBRE,CODIGO_TIPOLOGIA,TIPOLOGIA,CODIGO_CATEGORIA_TIPOLOGIA,CATEGORIA_TIPOLOGIA,CODIGO_DEPENDENCIA,DEPENDENCIA,ORIGEN_FINANCIAMIENTO,...,LOCALIDAD,CP,DOMICILIO,TE1,TE2,TE3,TE4,LATITUD,LONGITUD,ESTADO_GEO
0,50380842150137,NaN,CAPS EL TORO (17),ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Sin atención médica en forma periódica (menor ...,21,Provincial,Público,...,EL TORO,4411,AVENIDA LAVALLE ESQUINA 24 DE OCTUBRE,NaN,NaN,NaN,NaN,-23.208534,-66.827087,NaN
1,50380632150075,NaN,CAPS BARRO NEGRO (9) (EX LOTE ROSARIO DE RIO G...,ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Con atención médica general por lo menos 3 día...,21,Provincial,Público,...,LA MENDIETA,4522,CARLOS GARDEL S/N° - LOTE BARRO NEGRO,NaN,NaN,NaN,NaN,-24.308135,-64.931750,NaN
2,50260212329179,NaN,KIN SPORT,ESSIDT,Establecimiento de salud sin internación de di...,ESSDIT,Con atención médica diaria y con especialidade...,23,Privado,Privado,...,COMODORO RIVADAVIA,3240,Avenida Rivadavia 633,0297-4475310,NaN,NaN,NaN,-45.862127,-67.483555,NaN
3,51260352329290,NaN,LABORATORIO ANALISIS CLINICOS GEROSA,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,25 de Mayo 498,2945452050,294515503809,NaN,NaN,-42.915051,-71.319707,NaN
4,51260352329289,NaN,LABORATORIO ANALISIS CLINICOS,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,Ameghino 1033,02945-452155,NaN,NaN,NaN,-42.915771,-71.318034,NaN


<a class='anchor' id='ESPI'></a>
# refes_establecimientos_de_salud_publica_con_internacion_22-06-2018.xlsx

In [88]:
data = pd.read_excel('../data/raw/refes_establecimientos_de_salud_publica_con_internacion_22-06-2018.xlsx')

In [171]:
data.head()

,CODIGO,GLN,NOMBRE,CODIGO_TIPOLOGIA,TIPOLOGIA,CODIGO_CATEGORIA_TIPOLOGIA,CATEGORIA_TIPOLOGIA,CODIGO_DEPENDENCIA,DEPENDENCIA,ORIGEN_FINANCIAMIENTO,...,LOCALIDAD,CP,DOMICILIO,TE1,TE2,TE3,TE4,LATITUD,LONGITUD,ESTADO_GEO
0,50380842150137,NaN,CAPS EL TORO (17),ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Sin atención médica en forma periódica (menor ...,21,Provincial,Público,...,EL TORO,4411,AVENIDA LAVALLE ESQUINA 24 DE OCTUBRE,NaN,NaN,NaN,NaN,-23.208534,-66.827087,NaN
1,50380632150075,NaN,CAPS BARRO NEGRO (9) (EX LOTE ROSARIO DE RIO G...,ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Con atención médica general por lo menos 3 día...,21,Provincial,Público,...,LA MENDIETA,4522,CARLOS GARDEL S/N° - LOTE BARRO NEGRO,NaN,NaN,NaN,NaN,-24.308135,-64.931750,NaN
2,50260212329179,NaN,KIN SPORT,ESSIDT,Establecimiento de salud sin internación de di...,ESSDIT,Con atención médica diaria y con especialidade...,23,Privado,Privado,...,COMODORO RIVADAVIA,3240,Avenida Rivadavia 633,0297-4475310,NaN,NaN,NaN,-45.862127,-67.483555,NaN
3,51260352329290,NaN,LABORATORIO ANALISIS CLINICOS GEROSA,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,25 de Mayo 498,2945452050,294515503809,NaN,NaN,-42.915051,-71.319707,NaN
4,51260352329289,NaN,LABORATORIO ANALISIS CLINICOS,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,Ameghino 1033,02945-452155,NaN,NaN,NaN,-42.915771,-71.318034,NaN


<a class='anchor' id='RFES'></a>
# refes_registros_federal_de_establecimientos_de_salud_22-06-2018.xlsx

In [89]:
data = pd.read_excel('../data/raw/refes_registros_federal_de_establecimientos_de_salud_22-06-2018.xlsx')

In [172]:
data.head()

,CODIGO,GLN,NOMBRE,CODIGO_TIPOLOGIA,TIPOLOGIA,CODIGO_CATEGORIA_TIPOLOGIA,CATEGORIA_TIPOLOGIA,CODIGO_DEPENDENCIA,DEPENDENCIA,ORIGEN_FINANCIAMIENTO,...,LOCALIDAD,CP,DOMICILIO,TE1,TE2,TE3,TE4,LATITUD,LONGITUD,ESTADO_GEO
0,50380842150137,NaN,CAPS EL TORO (17),ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Sin atención médica en forma periódica (menor ...,21,Provincial,Público,...,EL TORO,4411,AVENIDA LAVALLE ESQUINA 24 DE OCTUBRE,NaN,NaN,NaN,NaN,-23.208534,-66.827087,NaN
1,50380632150075,NaN,CAPS BARRO NEGRO (9) (EX LOTE ROSARIO DE RIO G...,ESSIDT,Establecimiento de salud sin internación de di...,ESSIDT,Con atención médica general por lo menos 3 día...,21,Provincial,Público,...,LA MENDIETA,4522,CARLOS GARDEL S/N° - LOTE BARRO NEGRO,NaN,NaN,NaN,NaN,-24.308135,-64.931750,NaN
2,50260212329179,NaN,KIN SPORT,ESSIDT,Establecimiento de salud sin internación de di...,ESSDIT,Con atención médica diaria y con especialidade...,23,Privado,Privado,...,COMODORO RIVADAVIA,3240,Avenida Rivadavia 633,0297-4475310,NaN,NaN,NaN,-45.862127,-67.483555,NaN
3,51260352329290,NaN,LABORATORIO ANALISIS CLINICOS GEROSA,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,25 de Mayo 498,2945452050,294515503809,NaN,NaN,-42.915051,-71.319707,NaN
4,51260352329289,NaN,LABORATORIO ANALISIS CLINICOS,ESSID,Establecimiento de salud sin internación de di...,ESSID,Laboratorio de Análisis Clínicos,23,Privado,Privado,...,ESQUEL,9200,Ameghino 1033,02945-452155,NaN,NaN,NaN,-42.915771,-71.318034,NaN
